In [1]:
from transformers import AutoTokenizer, BitsAndBytesConfig
from peft import PeftConfig, AutoPeftModelForCausalLM
import torch

/root/miniconda3/envs/LLM/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


merge할 경우 성능 손실이 발생한다고 함 -> 어쩔 수 없음

`-` Merge Adapter and save pretrained model

In [2]:
adapter_name = "results/lr5e-6/checkpoint-450"
save_directory = "model/Zip-Llama-sft"

model = AutoPeftModelForCausalLM.from_pretrained(
    adapter_name,
    use_cache = True,
    dtype = torch.bfloat16,
    device_map = "cuda:0"
)

Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.29it/s]


In [4]:
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear

In [5]:
model = model.merge_and_unload()

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_use_double_quant = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = torch.bfloat16
)

adapter_name = "results/lr5e-6/checkpoint-450"
save_directory = "model/Zip-Llama-sft"

config = PeftConfig.from_pretrained(adapter_name)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    quantization_config = bnb_config,
    use_cache = True,
    dtype = torch.bfloat16,
    device_map = "cuda:0"
)
model = PeftModel.from_pretrained(model, adapter_name)

model = model.merge_and_unload().to("cuda:0")

Loading checkpoint shards: 100%|██████████| 4/4 [00:21<00:00,  5.28s/it]
/root/miniconda3/envs/LLM/lib/python3.12/site-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [7]:
model.save_pretrained(save_directory)

tokenizer = AutoTokenizer.from_pretrained(adapter_name)
tokenizer.save_pretrained(save_directory)

TypeError: Object of type dtype is not JSON serializable

`-` Saved model test

In [3]:
import os
import torch
import pandas as pd
from datasets import load_dataset
from tqdm.auto import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer

In [4]:
model_name = "model/Zip-Llama-sft"

model = AutoModelForCausalLM.from_pretrained(model_name, use_cache = True, device_map = "cuda:0", dtype = torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast = True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "left"

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.24s/it]


In [19]:
gen_ds = load_dataset("json", data_files = "data/dpo_resource.json")["train"]
output_path = os.path.join("data", f"dpo_dataset_generated.csv")

results = []
idx = 0

ith_inference = {"subject_id" : gen_ds[idx]["subject_id"]}
ith_inference["text"] = gen_ds[idx]["messages"][1]["content"]

for i in range(5):
    input_ids = tokenizer.apply_chat_template(
                    gen_ds[idx]["messages"],
                    add_generation_prompt=True,
                    return_tensors="pt"
    ).to(model.device)

    terminators = [tokenizer.eos_token_id]

    outputs = model.generate(
        input_ids,
        max_new_tokens=1024,
        eos_token_id=terminators,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=1.0,
        top_p = 0.95
    )

    response = outputs[0][input_ids.shape[-1]:]
    generation = tokenizer.decode(response, skip_special_tokens=True)
    ith_inference[f"Gen_{i}"] = generation

results.append(ith_inference)

In [20]:
results

[{'subject_id': 14711614,
  'text': " \r\nName:  ___                    Unit No:   ___\r\n \r\nAdmission Date:  ___              Discharge Date:   ___\r\n \r\nDate of Birth:  ___             Sex:   F\r\n \r\nService: OBSTETRICS/GYNECOLOGY\r\n \r\nAllergies: \r\nReglan / Compazine / Viramune\r\n \r\nAttending: ___\r\n \r\nChief Complaint:\r\nVeritgo\r\n \r\nMajor Surgical or Invasive Procedure:\r\nlumbar puncture\r\n\r\n \r\nHistory of Present Illness:\r\n___ yo G3P1 at 19w3d WGA presents to gyn triage with persistent \r\nvertigo x 2 days. Worse with changes in position and upon \r\nlooking down, but also present when closes eyes and head is at \r\nrest. Had nausea, but no emesis. Denies tinnitus. Hearing is \r\nWNL. Denies headache. Denies visual changes. Denies weakness / \r\nsensory changes / falls. Of note, HIV meds changed ___. Has had \r\nsimilar sx prior to med change, resulting in a fall. However, \r\nthose sx resolved\r\nspontaneously and are now recurring.  Denies fevers / chi

`-` 지랄남

In [10]:
import os
import torch
import pandas as pd
from datasets import load_dataset
from tqdm.auto import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel, PeftConfig

In [3]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_use_double_quant = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = torch.bfloat16
)

In [4]:
adapter_name = "results/lr5e-6/checkpoint-450"

config = PeftConfig.from_pretrained(adapter_name)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    quantization_config = bnb_config,
    use_cache = True,
    dtype = torch.bfloat16,
    device_map = "cuda:0"
)

model = PeftModel.from_pretrained(model, adapter_name)

tokenizer = AutoTokenizer.from_pretrained(adapter_name, use_fast = True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "left"

Loading checkpoint shards: 100%|██████████| 4/4 [00:19<00:00,  4.89s/it]


In [15]:
tokenizer = AutoTokenizer.from_pretrained(adapter_name, use_fast = True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "left"

In [16]:
gen_ds = load_dataset("json", data_files = "data/dpo_resource.json")["train"]
output_path = os.path.join("data", f"dpo_dataset_generated.csv")

results = []
idx = 0

ith_inference = {"subject_id" : gen_ds[idx]["subject_id"]}
ith_inference["text"] = gen_ds[idx]["messages"][1]["content"]

for i in range(5):
    input_ids = tokenizer.apply_chat_template(
                    gen_ds[idx]["messages"],
                    add_generation_prompt=True,
                    return_tensors="pt"
    ).to(model.device)

    terminators = [tokenizer.eos_token_id]

    outputs = model.generate(
        input_ids,
        max_new_tokens=1024,
        eos_token_id=terminators,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=1.0,
        top_p = 0.95
    )

    response = outputs[0][input_ids.shape[-1]:]
    generation = tokenizer.decode(response, skip_special_tokens=True)
    ith_inference[f"Gen_{i}"] = generation

results.append(ith_inference)

In [17]:
results

[{'subject_id': 14711614,
  'text': " \r\nName:  ___                    Unit No:   ___\r\n \r\nAdmission Date:  ___              Discharge Date:   ___\r\n \r\nDate of Birth:  ___             Sex:   F\r\n \r\nService: OBSTETRICS/GYNECOLOGY\r\n \r\nAllergies: \r\nReglan / Compazine / Viramune\r\n \r\nAttending: ___\r\n \r\nChief Complaint:\r\nVeritgo\r\n \r\nMajor Surgical or Invasive Procedure:\r\nlumbar puncture\r\n\r\n \r\nHistory of Present Illness:\r\n___ yo G3P1 at 19w3d WGA presents to gyn triage with persistent \r\nvertigo x 2 days. Worse with changes in position and upon \r\nlooking down, but also present when closes eyes and head is at \r\nrest. Had nausea, but no emesis. Denies tinnitus. Hearing is \r\nWNL. Denies headache. Denies visual changes. Denies weakness / \r\nsensory changes / falls. Of note, HIV meds changed ___. Has had \r\nsimilar sx prior to med change, resulting in a fall. However, \r\nthose sx resolved\r\nspontaneously and are now recurring.  Denies fevers / chi